In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from neuralnet import NeuralNet
from dataParser import DataParser
import torch


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/usr/local/Cellar/python@3.9/3.9.20/Frameworks/Python.framework/Versions/3.9/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/local/Cellar/python@3.9/3.9.20/Frameworks/Python.framework/Versions/3.9/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/Users/kjams/Desktop/meow_cmds/ai/lib/python3.9/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/kjams/Desktop/meow

In [2]:
db = DataParser('/Users/kjams/Desktop/meow_cmds/data/historical_data')

Loaded 21 files with 46110 rows.


In [3]:
df = db.df.dropna()
df.head()

,Price,Close,High,Low,Open,Volume,Sign,Symbol,Timestamp
2,2015-01-02,12.027410507202148,12.062921205605607,11.927980731023958,11.927980731023958,27226736,Dividend,T,2015-01-02
3,2015-01-05,11.913775444030762,12.045164751939703,11.903122862820174,12.027409852083789,40288128,Dividend,T,2015-01-05
4,2015-01-06,11.931536674499512,12.006109260722459,11.842760341274651,11.991904616962731,39184839,Dividend,T,2015-01-06
5,2015-01-07,12.001099586486816,12.153058108478962,11.841905640536185,12.149439939660223,41917840,Dividend,T,2015-01-07
6,2015-01-08,12.12049674987793,12.149441189726982,11.99386448290867,12.026427091948738,30186935,Dividend,T,2015-01-08


In [4]:
df['Symbol'].unique()

array(['T', 'BTC-USD', 'COIN', 'GME', 'AMC', 'SI=F', 'MMM', 'CL=F', 'KO',
       'RUN', 'BZ=F', 'GC=F', 'SOFI', 'MRNA', 'SNOW', 'SHOP', 'PLUG',
       'PFE', 'VZ', 'PLTR', 'ROKU'], dtype=object)

In [5]:
df = df.iloc[:, 1:]
df = df.drop(['Sign'], axis=1)

df.head()

,Close,High,Low,Open,Volume,Symbol,Timestamp
2,12.027410507202148,12.062921205605607,11.927980731023958,11.927980731023958,27226736,T,2015-01-02
3,11.913775444030762,12.045164751939703,11.903122862820174,12.027409852083789,40288128,T,2015-01-05
4,11.931536674499512,12.006109260722459,11.842760341274651,11.991904616962731,39184839,T,2015-01-06
5,12.001099586486816,12.153058108478962,11.841905640536185,12.149439939660223,41917840,T,2015-01-07
6,12.12049674987793,12.149441189726982,11.99386448290867,12.026427091948738,30186935,T,2015-01-08


In [6]:
df.iloc[:, 0:4] = df.iloc[:, 0:4].apply(
    pd.to_numeric, errors='ignore').round(2)
df.head()

/var/folders/px/rn3ttr855cb9h3ppww7dmvf40000gn/T/ipykernel_8922/3179097564.py:1: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df.iloc[:, 0:4] = df.iloc[:, 0:4].apply(


,Close,High,Low,Open,Volume,Symbol,Timestamp
2,12.03,12.06,11.93,11.93,27226736,T,2015-01-02
3,11.91,12.05,11.9,12.03,40288128,T,2015-01-05
4,11.93,12.01,11.84,11.99,39184839,T,2015-01-06
5,12.0,12.15,11.84,12.15,41917840,T,2015-01-07
6,12.12,12.15,11.99,12.03,30186935,T,2015-01-08


In [7]:


# 🚀 Step 1: Drop irrelevant columns

# 🚀 Step 2: Encode categorical columns
df['Symbol'] = df['Symbol'].astype('category').cat.codes  # Encode Sign

# 🚀 Step 3: Convert timestamp into numerical features
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df['Year'] = df['Timestamp'].dt.year
df['Month'] = df['Timestamp'].dt.month
df['Day'] = df['Timestamp'].dt.day
df = df.drop('Timestamp', axis=1)
df = df.drop('Volume', axis=1)

# 🚀 Step 4: Normalize continuous columns
continuous_cols = ['Close', 'High', 'Low', 'Open']
df[continuous_cols] = (df[continuous_cols] -
                       df[continuous_cols].mean()) / df[continuous_cols].std()

# 🚀 Step 5: Extract labels and features
X = df[['Symbol', 'Year', 'Month', 'Day']].values    # Inputs
y = df[['Close', 'High', 'Low', 'Open']].values  # Outputs

# 🚀 Step 6: Convert to tensors
features = torch.tensor(X, dtype=torch.float32)
labels = torch.tensor(y, dtype=torch.float32)

In [8]:
df.head()

,Close,High,Low,Open,Symbol,Year,Month,Day
2,-0.208408,-0.208336,-0.208534,-0.208452,19,2015,1,2
3,-0.208423,-0.208337,-0.208538,-0.20844,19,2015,1,5
4,-0.20842,-0.208342,-0.208545,-0.208445,19,2015,1,6
5,-0.208412,-0.208326,-0.208545,-0.208426,19,2015,1,7
6,-0.208397,-0.208326,-0.208527,-0.20844,19,2015,1,8


In [9]:
nn = NeuralNet(architecture='wide')

/Users/kjams/Desktop/meow_cmds/ai/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


In [10]:
nn.train_model(X, y)

🚀 Starting training...
🔥 Epoch [1/20] Loss: 0.7724
✅ Validation Loss: 0.6292
✅ New best model saved (epoch 1)
🔥 Epoch [2/20] Loss: 0.6733
✅ Validation Loss: 0.2705
✅ New best model saved (epoch 2)
🔥 Epoch [3/20] Loss: 0.4757
✅ Validation Loss: 0.2203
✅ New best model saved (epoch 3)
🔥 Epoch [4/20] Loss: 0.4408
✅ Validation Loss: 0.6865
🔥 Epoch [5/20] Loss: 0.4330
✅ Validation Loss: 0.1628
✅ New best model saved (epoch 5)
🔥 Epoch [6/20] Loss: 0.4056
✅ Validation Loss: 0.0776
✅ New best model saved (epoch 6)
🔥 Epoch [7/20] Loss: 0.5171
✅ Validation Loss: 0.1612
🔥 Epoch [8/20] Loss: 0.3750
✅ Validation Loss: 0.0962
🔥 Epoch [9/20] Loss: 0.4244
✅ Validation Loss: 0.5698
⏸️ Early stopping triggered. No improvement for 3 epochs.
🚀 Training complete.


In [ ]:
deep_wide_model = NeuralNet()
deep_wide_model.model = torch.load('deep_wide_model.pth')

/Users/kjams/Desktop/meow_cmds/ai/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


In [20]:
x = [19, 2025, 4, 12]
tensor_x = torch.tensor(x, dtype=torch.float32)

nn.predict(tensor_x)

tensor([1.1804, 1.1641, 1.1997, 1.1858])